In [33]:
import os

In [34]:
%pwd

'c:\\Users\\sanja\\OneDrive\\Documents'

In [35]:
os.chdir("../")

In [36]:
%pwd

'c:\\Users\\sanja\\OneDrive'

In [37]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [38]:
from cnnClassifier.constants import *
from pathlib import Path 
from cnnClassifier.utils.common import read_yaml, create_directories

In [39]:
from pathlib import Path
import os  # Add import for os if not already imported

CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")

class ConfigurationManager:
    ABSOLUTE_CONFIG_FILE_PATH = Path("C:/Users/sanja/OneDrive/Documents/GitHub/Chicken-disease-classification/config/config.yaml")
    ABSOLUTE_PARAMS_FILE_PATH = Path("C:/Users/sanja/OneDrive/Documents/GitHub/Chicken-disease-classification/params.yaml")
    
    def __init__(self, config_filepath=ABSOLUTE_CONFIG_FILE_PATH, params_filepath=ABSOLUTE_PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self):
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config


In [40]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [41]:
from pathlib import Path
import os
import time
import tensorflow as tf

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = Path(self.config.tensorboard_root_log_dir) / f"tb_logs_at_{timestamp}"
        return tf.keras.callbacks.TensorBoard(log_dir=str(tb_running_log_dir))
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]



In [42]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

[2024-02-17 22:13:53,576: INFO: common: yaml file: C:\Users\sanja\OneDrive\Documents\GitHub\Chicken-disease-classification\config\config.yaml loaded successfully]
[2024-02-17 22:13:53,578: INFO: common: yaml file: C:\Users\sanja\OneDrive\Documents\GitHub\Chicken-disease-classification\params.yaml loaded successfully]
[2024-02-17 22:13:53,579: INFO: common: created directory at: artifacts]
[2024-02-17 22:13:53,580: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-02-17 22:13:53,581: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]


In [43]:
import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
f"tb_logs_at_{timestamp}"

'tb_logs_at_2024-02-17-22-13-53'